## Qu'est-ce que les Prototypes?

En JavaScript, chaque objet a un lien caché vers un autre objet appelé son **prototype**. Lorsque vous essayez d'accéder à une propriété sur un objet, JavaScript cherche d'abord dans l'objet lui-même. S'il ne la trouve pas, il cherche dans le prototype de l'objet, puis dans le prototype du prototype, et ainsi de suite - formant une **chaîne de prototypes**.

Pensez-y comme à l'héritage dans un arbre généalogique: si vous n'avez pas quelque chose, vous pouvez l'emprunter à votre parent, et s'il ne l'a pas, à son parent, etc.

Nous pouvons explorer cela avec ```Object.getPrototypeOf(obj)```

In [ ]:
// Всеки обект има прототип
let obj = { a: 1 };
console.log(Object.getPrototypeOf(obj));  // Object.prototype

// Масивите имат Array.prototype
let arrr = [1, 2, 3];
console.log(Object.getPrototypeOf(arrr));  // Array.prototype

// Функциите имат Function.prototype
function fn() {}
console.log(Object.getPrototypeOf(fn));   // Function.prototype

# Les Prototypes et l'Héritage en JavaScript

Comprendre le modèle d'héritage basé sur les prototypes de JavaScript.

## 1. Exemples d'Object.create()

Avant de plonger dans les prototypes, explorons `Object.create()` et la manipulation des prototypes - des concepts fondamentaux pour comprendre le modèle d'héritage de JavaScript.

### Manipulation Basique des Prototypes

**Note:** Dans Deno (que ce notebook utilise), l'assignation directe à `__proto__` ne fonctionne pas comme prévu. La bonne façon de définir le prototype d'un objet est d'utiliser `Object.setPrototypeOf()`, qui fonctionne de manière cohérente dans Node.js et Deno.

In [ ]:
let obj = { cook: "Baba", meal: "Salad" };
let othObj = { value: 10 };

// Начин за определяне на прототип (работи в Node.js и Deno)
Object.setPrototypeOf(othObj, obj);

console.log(othObj.value);  // 10 (собствено свойство)
console.log(othObj.cook);   // "Baba" (наследено от obj)
console.log(othObj.meal);   // "Salad" (наследено от obj)

// Забележка: В Node.js, othObj.__proto__ = obj също работи, но е остаряло
// и не работи в строг режим на Deno

### Разширяване на Вградени Прототипи (Използвайте с Осторожност!)

In [ ]:
// Добавяне на метод към Array.prototype
Array.prototype.oddCnt = function () {
    return this.filter( el => el % 2).length;
};

console.log([].oddCnt.call([1, 2, 3, 5, 6, 7]));  // 4 (четири нечетни числа)

### Object.create() avec les Descripteurs de Propriété

In [ ]:
let grandp = {
    name: "Baba",
    age: 105,
};

// Créez un objet avec grandp comme prototype et définissez les propriétés
const parent = Object.create(grandp, {
    name: {
        writable: true,
        configurable: true,
        value: "Kitty",
    },
});

console.log(parent.name);  // "Kitty" (propriété propre)
console.log(parent.age);   // 105 (hérité de la grand-mère)

### Object.create() avec les Propriétés de Données et d'Accesseur

In [ ]:
let o = Object.create(Object.prototype, {
    
    foo: {      // foo е обикновено свойство на данни
        writable: true,
        configurable: true,
        value: "hello",
    },

    bar: {      // bar е свойство за достъп
        configurable: false,
        get() { // <<------  има getter
            return 10;
        },
        set(value) { // ...  и setter
            console.log("Setting `o.bar` to", value);
        },
    },
});

console.log(o.foo);  // "hello"
console.log(o.bar);  // 10
o.bar = 42;          // "Setting `o.bar` to 42"

### Inspection et Modification des Descripteurs de Propriété

JavaScript fournit des méthodes pour inspecter et modifier les caractéristiques des propriétés (descripteurs) des objets.

In [ ]:
// Object.getOwnPropertyDescriptors() - voir TOUS les descripteurs de propriété
let objX = { x: 10, y: 20, z: 33, value: "yes" };
objX.arr = [1, 23];

console.log("Всички дескриптори:");
console.log(Object.getOwnPropertyDescriptors(objX));
// {
//   x: { value: 10, writable: true, enumerable: true, configurable: true },
//   y: { value: 20, writable: true, enumerable: true, configurable: true },
//   z: { value: 33, writable: true, enumerable: true, configurable: true },
//   value: { value: "yes", writable: true, enumerable: true, configurable: true },
//   arr: { value: [ 1, 23 ], writable: true, enumerable: true, configurable: true }
// }

In [ ]:
// Object.getOwnPropertyDescriptor() - получава дескриптор за ЕДНО свойство
console.log("Дескриптор за 'z':");
console.log(Object.getOwnPropertyDescriptor(objX, "z"));
// { value: 33, writable: true, enumerable: true, configurable: true }

In [ ]:
// Object.getOwnPropertyNames() - връща ВСИЧКИ собствени свойства (включително неперечислими)
console.log("Всички имена на свойства:", Object.getOwnPropertyNames(objX));
// [ "x", "y", "z", "value", "arr" ]

In [ ]:
// Object.defineProperty() - модифицира дескриптор на свойство

// Направи свойството 'z' неперечислимо (скрито от for...in цикли)
Object.defineProperty(objX, "z", { enumerable: false, value: objX.z });

console.log("След направяне на 'z' неперечислимо:");
console.log(objX);  // z е скрит от нормално изписване
// { x: 10, y: 20, value: "yes", arr: [ 1, 23 ] }

console.log("Но 'z' все още съществува:");
console.log(objX.z);  // 33

In [ ]:
// Направи свойството само за четене (неписуемо)
Object.defineProperty(objX, "x", { writable: false, value: objX.x });

console.log("След направяне на 'x' само за четене:");

try {
  objX.x = 499;  // хвърля грешка в строг режим
} catch (e) {
  console.log("Грешка:", e.message);
  // Грешка: Cannot assign to read only property 'x' of object
}

### Функция Конструктор която Работи със или без 'new'

In [ ]:
function Cons() { // Тази функция поддържа конструирането както
    let self;     // с new контекст, така и без, макар че не се препоръчва

    if (this === global || this === undefined) {
        self = Object.create(Cons.prototype);
    } else {
        self = this;
    }

    self.baba = 10;
    return self;
}

let obj1 = new Cons();
let obj2 = Cons();

console.log(obj1.baba);  // 10
console.log(obj2.baba);  // 10
console.log(obj1 instanceof Cons);  // true
console.log(obj2 instanceof Cons);  // true

### Modèle d'Héritage utilisant Object.create()

In [ ]:
function BaseClass() {
    return this;
}

BaseClass.prototype.val = 10;

function InheriClass() {
    return this;
}

// Установи наследяване с помощта на Object.create()
// Това позволява на верижката от прототипи да се следи
// където това по същество казва нещо като
// InheriClass.prototype = { __proto__ : BaseClass.prototype }
InheriClass.prototype = Object.create(BaseClass.prototype);

let v = new InheriClass();
console.log(v.val);  // 10 (наследено от BaseClass)

## 2. La Chaîne de Prototypes (ELI5)

Imaginez que vous cherchez un jouet:

1. D'abord, vous vérifiez votre propre boîte à jouets

2. Si ce n'est pas là, vous demandez à votre frère aîné

3. S'ils ne l'ont pas, ils demandent à vos parents

4. Si les parents ne l'ont pas, ils demandent aux grands-parents

5. Finalement, vous atteignez la fin (null)

C'est exactement comment fonctionne la chaîne de prototypes de JavaScript!

In [ ]:
let grandparent = { surname: "Smith" };
let myparent = Object.create(grandparent);
myparent.job = "Engineer";
let child = Object.create(myparent);
child.name = "Alice";

console.log(child.name);     // "Alice" (собствено свойство)
console.log(child.job);      // "Engineer" (от myparent)
console.log(child.surname);  // "Smith" (от grandparent)

// Верижката: child -> parent -> grandparent -> Object.prototype -> null

## 3. Fonctions Constructeurs et Prototypes

In [ ]:
function Person(name, age) {
    this.name = name;
    this.age = age;
}

// Добави методи към прототип (споделени от всички екземпляри)
Person.prototype.greet = function() {
    return `Hi, I'm ${this.name}`;
};

const alice = new Person("Lisa", 25);
const bob = new Person("Zugg", 30);

console.log(alice.greet());     // "Hi, I'm Lisa"
console.log(bob.greet());       // "Hi, I'm Zugg"

// И двамата споделят един и същи метод greet
console.log(alice.greet === bob.greet);  // true

## 4. Comprendre `__proto__` vs `prototype`

- `prototype`: Propriété sur les fonctions constructeurs, définit ce que les instances hériteront

- `__proto__`: Propriété sur les objets, pointe vers l'objet prototype réel

Pensez-y de cette façon:

- `prototype` est le plan

- `__proto__` est la connexion réelle

In [ ]:
function Dog(name) {
    this.name = name;
}

Dog.prototype.bark = function() {
    return "Woof!";
};

const rex = new Dog("Rex");

console.log(Dog.prototype);     // Dog.prototype е планът

// rex.__proto__ по същество сочи към Dog.prototype
// но може да бъде достъпен директно само в NodeJS
// където в Deno има ограничение да го докоснем

console.log(Object.getPrototypeOf(rex) === Dog.prototype);  // true

// rex може да използва bark поради верижката от прототипи
console.log(rex.bark());  // "Woof!"

## 5. Object.create() et Manipulation des Prototypes

In [ ]:
// Object.create() създава обект със специфичен прототип
const animal = {
    eat:   function() {    return "Eating...";    }
};

const dog = Object.create(animal);
dog.bark = function() {    return "Woof!";        };

console.log(dog.bark());  // "Woof!" (собствен метод)
console.log(dog.eat());   // "Eating..." (от прототип)

console.log(Object.getPrototypeOf(dog) === animal);  // true

## 6. Modèles d'Héritage

In [ ]:
// Функция родител
function Animal(name) {
    this.name = name;
}

Animal.prototype.eat = function() {
    return this.name + ' is eating';
};

function Dog(name, breed) {   // Функция дете
    Animal.call(this, name);  // Извикай родителския конструктор
    this.breed = breed;       // но използвай контекста на Dog (this)
}

if ( false ) {   
    // някои източници правят наследяване по този начин
    // но никога не поправят конструктора, който ще
    // сочи към функцията Animal, ако го оставим както е
    Dog.prototype = Object.create(Animal.prototype);

    // така че също трябва да поправим препратката на конструктора, тъй като
    // новосъздаденият прототип обект няма свойство constructor
    // и ще се разреши към свойството Animal.constructor
    Dog.prototype.constructor = Dog; 
} else {
    // алтернативно, можем просто да кажем, което е също по-ясно
    Object.setPrototypeOf(Dog.prototype, Animal.prototype);
}

Dog.prototype.bark = function() {
    return this.name + ' says Woof!';
};

const rex = new Dog("Rex", "German Shepherd");
console.log(rex.eat());   // "Rex is eating" (наследено)
console.log(rex.bark());  // "Rex says Woof!" (собствен метод)
console.log(Object.getPrototypeOf(rex).constructor)

### Exemple de Chaîne de Prototypes Visuelle

```
+--------------------+
|   rex (instance)   |
|--------------------|  
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|   Dog.prototype    |
|--------------------|  
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Animal.prototype  |
|--------------------|  
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Object.prototype  |
|--------------------|  
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|        null        |
+--------------------+
```

In [ ]:
// Проверка на верижката
console.log(Object.getPrototypeOf(rex) === Dog.prototype);  // true
console.log(Object.getPrototypeOf(Dog.prototype) === Animal.prototype);  // true
console.log(Object.getPrototypeOf(Animal.prototype) === Object.prototype);  // true
console.log(Object.getPrototypeOf(Object.prototype) === null);  // true

### Classes ES6 (Sucre Syntaxique)

Les classes ES6 ne sont que du sucre syntaxique pour les prototypes - elles fonctionnent de la même manière!

In [ ]:
class ModernAnimal {
    constructor(name) {
        this.name = name;
    }
    
    eat() {
        return `${this.name} is eating light`;
    }
}

class FancyDog extends ModernAnimal {
    constructor(name, breed) {
        super(name);          // Извикай родителския конструктор
        this.breed = breed;
    }
    
    bark() {
        return this.name + 'says Zap Zap!';
    }
}

const rex = new FancyDog( "Rex", "German Cybernaut");
console.log(rex.eat());       // "Rex is eating"
console.log(rex.bark());      // "Rex says Woof!"

// Все още използва прототипи под капака
console.log(Object.getPrototypeOf(rex) === FancyDog.prototype);  // true
console.log(rex.constructor, Object.getPrototypeOf(rex).constructor);

###  Mixins

Les mixins sont un moyen d'ajouter des fonctionnalités aux objets ou aux classes sans utiliser l'héritage. Un mixin est un objet qui contient des méthodes à utiliser par d'autres objets. Vous pouvez utiliser `Object.assign()` pour copier les méthodes du mixin au prototype d'une fonction constructeur ou d'une classe.

In [ ]:
const flyMixin = {
  fly() {
    console.log(`${this.name} is flying!`);
  }
};

function Bird(name) {
  this.name = name;
}

Object.assign(Bird.prototype, flyMixin);

const eagle = new Bird('Eagle');
eagle.fly(); // Eagle is flying!

## 7. Résumé

- **Les Prototypes** permettent l'héritage d'objets en JavaScript

- **La chaîne de prototypes** est la façon dont JavaScript recherche les propriétés

- **Les fonctions constructeurs** utilisent `prototype` pour partager des méthodes

- **`__proto__`** est le lien réel, **`prototype`** est le plan

- **Les classes ES6** ne sont que du sucre syntaxique sur les prototypes

- Comprendre les prototypes est essentiel pour maîtriser JavaScript!

## 8. Pièges et Bonnes Pratiques

Bien que les prototypes soient puissants, il existe certains pièges courants et bonnes pratiques à garder à l'esprit.

### a) Ne Modifiez Jamais `Object.prototype`

La modification de `Object.prototype` est considérée comme une mauvaise pratique car elle affecte tous les objets de votre application. Cela peut entraîner un comportement inattendu et des conflits avec d'autres bibliothèques. C'est souvent appelé "prototype pollution".

In [ ]:
// Mauvaise pratique!
Object.prototype.foo = 'bar';

const myObj = {};
console.log(myObj.foo); // 'bar'

// Cela peut casser les boucles for...in
for (const key in myObj) {
    console.log(key); // 'foo'
}

### b) Réinitialiser la Propriété `constructor`

Lorsque vous définissez manuellement le prototype d'une fonction constructeur, vous écrasez sa propriété `constructor`. Il est important de la réinitialiser pour qu'elle pointe vers la fonction constructeur correcte. Cela garantit que `instanceof` et d'autres mécanismes qui dépendent de la propriété constructor fonctionnent correctement. Ou utilisez `Object.setPrototypeOf` à la place.

In [ ]:
function Animal() {}
function Dog() {}

Dog.prototype = Object.create(Animal.prototype);
// Sans la ligne suivante, rex.constructor serait Animal
Dog.prototype.constructor = Dog;

// ou utilisez
Object.setPrototypeOf(Dog.prototype, Animal.prototype);

const rex = new Dog();
console.log(rex.constructor === Dog); // true

## Exercices

### Exercice 1: Réparez la Chaîne de Prototypes Cassée

Le code suivant a une chaîne de prototypes cassée. Réparez-le afin que `cat.purr()` et `cat.eat()` fonctionnent tous les deux.

In [ ]:
function Animal() { }
Animal.prototype.munch = function() { console.log('munching...') };

function Cat() { }
// Réparez la ligne ci-dessous
Cat.prototype = new Animal();

Cat.prototype.purr = function() { console.log('purrrr') };

const cat = new Cat();
cat.purr();
cat.munch();

### Exercice 2: Implémentez un `SuperArray`

Créez un constructeur `SuperArray` qui hérite d'`Array`. Ajoutez une méthode au prototype `SuperArray` appelée `last()` qui retourne le dernier élément du tableau.

In [ ]:
function SuperArray(...args) {
  const arr = new Array(...args);
  Object.setPrototypeOf(arr, SuperArray.prototype);
  return arr;
}

SuperArray.prototype = Object.create(Array.prototype);

// Ajoutez votre méthode ici

const myArr = new SuperArray(1, 2, 3, 4, 5);
console.log(myArr.last()); // 5